Attention is all your need 这篇文章提出的原始的transform 依旧是属于seq2seq模型的，后面的via bert 这些模型把transform肢解了，只取了其中一部分

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [21]:
torch.manual_seed(99)

## word enbedding

In [39]:
batch_size = 2
# 单词表 大小
max_num_src_words = 8
max_num_tgt_words = 8

# 序列的最大长度
max_src_seq_len = 5
max_tgt_seq_len = 5
# 维度
model_dim = 8

# 各个样本的长度
# src_len = torch.randint(2, 5, size=(batch_size,))
# tgt_len = torch.randint(2, 5, size=(batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)
src_len, tgt_len

(tensor([2, 4], dtype=torch.int32), tensor([4, 3], dtype=torch.int32))

In [28]:
# 生成源序列，单词的索引 （从 1 开始， 0 留给 pad
src_seq = [torch.randint(1, max_num_src_words, (L,)) for L in src_len]
tgt_seq = [torch.randint(1, max_num_tgt_words, (L,)) for L in tgt_len]

src_seq, tgt_seq

([tensor([7, 7]), tensor([6, 3, 6, 6])],
 [tensor([3, 1, 2, 4]), tensor([1, 7, 1])])

In [38]:
# 扩充原序列和 目标序列
src_seq = torch.cat(
            [torch.unsqueeze(
                            F.pad(s, (0, max_src_seq_len-len(s))),    # 扩充
                            0)                                        # 拓展为 2 维度
            for s in src_seq]
            )

tgt_seq = torch.cat(
            [torch.unsqueeze(
                            F.pad(s, (0, max_tgt_seq_len-len(s))),    # 扩充
                            0)                                        # 拓展为 2 维度
            for s in tgt_seq]
            )

src_seq, tgt_seq

(tensor([[7, 7, 0, 0, 0],
         [6, 3, 6, 6, 0]]),
 tensor([[3, 1, 2, 4, 0],
         [1, 7, 1, 0, 0]]))

In [42]:
# 构造embedding
# 这里是没有经过学习的随机参数，正常使用是需要 经过训练的
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)

src_embedding_table.weight # 每一行代表一个 embedding 向量

Parameter containing:
tensor([[-0.3234, -0.1763, -0.2570,  1.3236, -0.1547,  1.4877,  0.3349, -2.0956],
        [ 0.5956, -0.7098, -0.4343,  1.5765, -0.0614,  1.0428,  0.2707,  0.4077],
        [ 1.8377, -1.6478, -1.0745,  0.2456, -0.7835,  0.7687,  1.8743, -0.2222],
        [ 0.2401, -1.6211,  1.6115,  0.5610, -0.6752, -1.5611, -1.5528, -2.3721],
        [ 0.9949,  0.6403,  1.9627, -1.9418,  1.0155,  0.1313,  0.5024,  1.4581],
        [ 1.5008, -1.4159, -1.9909,  2.0253, -1.6323,  1.6446, -0.8791, -0.8754],
        [ 0.0454, -0.2379,  0.6980,  0.1194,  0.2561,  0.8265,  0.1867, -0.0045],
        [ 1.4138, -0.1725, -0.0856, -0.5033, -2.1081, -0.3378, -0.2584, -0.1200],
        [-1.4306,  0.1935, -1.8466,  0.1552,  0.9443,  0.8908,  0.3843, -0.4270]],
       requires_grad=True)

In [45]:
src_embedding = src_embedding_table(src_seq)
tgt_embedding = tgt_embedding_table(tgt_seq)

src_embedding.shape # batch_szie, seq_len, features

torch.Size([2, 5, 8])

## position_embedding
本质上 构建一个 seq_len * features 的 矩阵，每一行代表一个时刻的位置信息，位置信息要能满足：能保留时序数据的相对位置信息和绝对位置信息。

$$PE_{pos, 2i} = sin(pos/ 100000 ^{2i/d_{model}})$$
$$PE_{pos, 2i+1} = cos(pos/ 100000 ^{2i/d_{model}})$$

pos 代表某一时刻， i 表示某时刻下第i个维度

transform正余弦函数来编码，在偶数维度用 sin表示,奇数维度用 cos 表示。
具体做法是
1. 先构建两个如下矩阵：
    + seq_len * 1 的 pos_matrix: [[1], [2], [3], [...]] 
    + 1 * model_d // 2 的 i_matrix 矩阵：[[0, 2, 4, 6, ..., model_d]]
2. 然后对 i_matrix 矩阵做 指数为 100000 的 指数运算，
3. 最后分别对奇数列 偶数列 执行cos sin 操作

In [57]:
# 定义编码最大长度
max_position_len = 5

# pos 表示行， 每行代表序列的一个时刻 位置编码, 
# max_position_len 表示有多少个位置需要编码
pos_mat = torch.arange(max_position_len).reshape((-1, 1)) 
# i  表示列， 具体就是 第几个特征
i_mat = torch.arange(0,model_dim,2).reshape((1, -1)) / model_dim
i_mat = torch.pow(10000, i_mat)


# 行表示 时刻，列表示 该时刻的第几维特征
pe_embedding_table = torch.zeros(max_position_len, model_dim)
# 偶数列
pe_embedding_table[:, 0::2] = torch.sin(pos_mat / i_mat)
# 奇数列
pe_embedding_table[:, 1::2] = torch.cos(pos_mat / i_mat)

print("pe_embedding_table", pe_embedding_table)

# 把这个 table 做为 enbedding 的参数
pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
print("pe_embedding.weight", pe_embedding.weight)

# 然后就可以这样使用了，此处应该   传入位置信息

#  src_len 是每个样本的最大长度，在一个batch中要改取最大的
src_pos = torch.cat([torch.arange(max(src_len)) for  _ in src_len]).to(torch.int32)
tgt_pos = torch.cat([torch.arange(max(tgt_len)) for  _ in tgt_len]).to(torch.int32)

src_pe_embedding = pe_embedding(src_pos)
tgt_pe_embedding = pe_embedding(tgt_pos)

pe_embedding_table tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
          9.9980e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
          9.9955e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.8942e-01,  9.2106e-01,  3.9989e-02,
          9.9920e-01,  4.0000e-03,  9.9999e-01]])
pe_embedding.weight Parameter containing:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
   

## scaled-dot-product attention

编码器是 对序列的每个位置 要计算出一个新的表征，这个表征是要包含之前所有时刻（或者之后时刻）的输入信息的。

transform 中的编码器核心是self-attention，做法是当前时刻的表征等于输入序列每个时刻的信息的加权求和，这就涉及到一个权重了，一个时刻对应着一个权重分布，所以不同时刻的表征信息是不同的。

权重具体是这样计算的：以当前时刻的信息得到查询向量Q，去和输入序列所有时刻的key计算相似度，再把相似度归一化，即得到了权重分布。归一化方式一般都是softmax。

Transform中相似度的计算方式如下：
$$\text{Attention}(Q, K, V) = softmax\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

这里的$Q$,$K$,$V$ 维度是一样的，都是 batch_size * seq_len * features, 计算的方式也是一样的，即每个位置的qkv是由该位置的输入信息 经过线性转换得到的。最后拼接为 QKV，
$QK^T$后的维度为 batch_size * seq_len * seq_len, 抛开batch_size不看，每一行都是一个时刻的注意力分数，然后和$V$相乘，得到该时刻的表征。

注：
+ $QK^T$ 计算得到shape为 batch_size * seq_len * seq_len, 第一个seq_len 表示输入的各个时刻t， 第二个 seq_len 表示 t时刻的输入 和 别的时刻的输入的相似度。 
+ $QK^T$ 可以理解为相似度矩阵, 当前时刻 和别的时刻的相似性


------
计算相似度的时候，为什么要做 scaled 呢? 主要是使得softmax出来的各个概率值之间方差不要太大（即概率分布不要太尖锐），因为方差太大，在softmax反向传播的时候，容易梯度消失。

举个例子：
```python
t_score = torch.randn(5)
alpha_1 = 0.1
alpha_2 = 10
prob_1 = F.softmax(t_score * alpha_1, -1)
prob_2 = F.softmax(t_score * alpha_2, -1)
prob_1, prob_2

(tensor([0.1973, 0.1893, 0.1777, 0.2027, 0.2328]),
 tensor([6.5025e-08, 1.0364e-09, 1.8629e-12, 9.7313e-07, 1.0000e+00]))
```
可以看见 prob_2分布的各个概率值之间方差很大。

但是这和维度有什么关系呢？

当维度很大的时候，得到的未经过softmax的每个分数就很大，因为是点积，维度越大，相乘的项就越多，所以得到的每个分数值就越大。
分数越大，正如 计算 prob_2 的时候，对每个分数 乘 10， 归一化之后的概率方差就会很大！

工程上看：
随着 $d_k$ 的增大，$q \cdot k$ 点积后的结果也随之增大，归一化之后的概率方差会很大，这样会将 softmax 函数推入梯度非常小的区域，使得收敛困难(可能出现梯度消失的情况)

从数学角度理解：
假设$q$和$k$的分量是具有均值$0$和方差$1$的独立随机变量，那么它们的点积 $q \cdot k=\sum_{i=1}^{d_k} q_{i} k_{i}$  均值为0，方差为 $d_k$，因此为了抵消这种影响，我们将点积缩放 $\frac{1}{\sqrt{d_{k}}}$

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask_matrix):
    """Q,K,V [batch_size*num_head, seq_len, model_dim / num_head]
    """
    
    score = torch.bmm(Q, K.transpose(-2, -1) / torch.srqt(model_dim)) # model_dim是编码的维度
    masked_score = score.masked_fill(mask_matrix, -np.inf) # [batch_size, seq_len, seq_len]
    prob = F.softmax(masked_score, -1) # [batch_size, seq_len, seq_len]
    context = torch.bmm(prob, V) # [batch_size, seq_len, features]
    return context

## 编码器的mask

构造 encoder 的 mask 矩阵:padding_mask

因为输入的每个样本长度不一致，我们要对输入序列进行对齐。具体来说，就是给在较短的序列后面填充 0，因为这些填充的位置，其实是没什么意义的，所以我们的attention机制不应该把注意力放在这些位置上，所以我们需要进行一些处理。

具体的做法是，在计算好相似度矩阵$QK^T$之后，把相似度矩阵中无效的位置赋值为一个非常大的负数(负无穷)，这样的话，经过 softmax，这些位置的概率就会接近0，在加权求和的时候，就不会把这些位置对应的$v$加进去了

所以做mask的位置在softmax之前，即计算完 $QV^T$之后，对这个相似度矩阵进行操作。

QV^T的shape 为 batch_size * seq_len * seq_len, 所以我们的mask矩阵也是 同样大小的。

(mask 有点像邻接矩阵，抛开batch_size，因为样本之间是没用关系的， 我们关心的是一个样本中，i时刻和j时刻之间的关系，更具体的，我们关心 QV^T 计算得到的分数中， 时刻i对时刻j的关系是否是有意义的，即i时刻或者j时刻是否是填充得到，如果是填充得到的，就没用意义)

In [80]:
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(
                                                            F.pad(
                                                                torch.ones(L), 
                                                                (0, max_src_seq_len-L)),
                                                            0) 
                                                for L in src_len]),
                                    2) 
# 每个样本的有效时刻
print("每个样本的有效时刻\n", valid_encoder_pos)

valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1,2))
print("邻接矩阵：\n", valid_encoder_pos_matrix)

# 计算相似度矩阵中无效的值
invalid_encoder_pos_matrix = 1 - valid_encoder_pos_matrix
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)

每个样本的有效时刻
 tensor([[[1.],
         [1.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [0.]]])
邻接矩阵：
 tensor([[[1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0.]]])


In [82]:
# 测试
# 随机生成一个 注意力分数
score = torch.randn(batch_size, max_src_seq_len, max_src_seq_len)
print("初始 注意力分数\n", score)

# 执行mask, 即对无效的相似值填充 负无穷的值
masked_score = score.masked_fill(mask_encoder_self_attention, -np.inf)
print("mask后 注意力分数:\n", masked_score)

prob = F.softmax(masked_score, -1)
print("概率值", prob)

初始 注意力分数
 tensor([[[-0.1391, -0.5662, -0.5094,  1.1162,  0.9927],
         [-0.2235,  0.4971, -0.3573,  0.7820,  0.1446],
         [-0.9438,  1.1743,  0.0848, -0.4829, -0.6657],
         [-0.1452, -1.1441,  0.7936,  1.4197, -1.0676],
         [ 0.6350,  0.4665, -0.2280, -0.2060,  0.2568]],

        [[-0.5949,  0.3144, -0.2984,  0.8045, -0.7115],
         [-0.6484,  1.2937, -0.2162, -1.4444, -0.1637],
         [-1.0035, -1.2722, -0.9794, -1.2821,  1.5584],
         [-0.0973,  1.2870,  0.6858,  0.5425, -0.7797],
         [ 1.4631,  0.8461, -0.3328, -0.2684,  0.2218]]])
mask后 注意力分数:
 tensor([[[-0.1391, -0.5662,    -inf,    -inf,    -inf],
         [-0.2235,  0.4971,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,    -inf,    -inf],
         [   -inf,    -inf,    -inf,    -inf,    -inf]],

        [[-0.5949,  0.3144, -0.2984,  0.8045,    -inf],
         [-0.6484,  1.2937, -0.2162, -1.4444,    -inf],
         [-1.0035,

## Decoder mask

transform 解码器包括两组不同的attention:
+ self-attention 在每个decoder-block的第一层，是计算当前时刻和之前时刻输入的相似度，以计算当前时刻的输出。需要保证因果性，即不能看到后面时刻的信息。只有在训练的时候需要，因为训练的时候采用teach-forceing，即把真实的序列全作为解码器的输入，这样如果不加干预，在计算selfattention的时候就能看到未来时刻的真实输入，显然是不合理的。在预测的时候，是不需要这部分的，因为预测的时候，是一步一步的预测出来的。因为第i步能看到i步及之前的序列，所以相似度矩阵的下三角是有效的，因此需要把上三角填充为负无穷。但由于每个样本的长度不一样，也涉及到填充0，所以每个样本的mask依旧是不一样的。
+ cross-attention 在每个decoder-block的第二层, 是计算前一时刻的输出和编码器各个输出表征之间的相似度。以得到当前时刻的输入。编码器输出是会补0的，所以此时需要掩码

### Decoder cross-attention mask

掩码都是针对相似度矩阵$QK^T$的，这里的$Q$是解码器当前时刻的输出计算得到的，$KV$是由编码器的输出计算得到的。
此时相似度矩阵 shape 为 [batch_size, tgt_seq_len, src_seq_len]

In [86]:
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(
                                                            F.pad(
                                                                torch.ones(L), 
                                                                (0, max_src_seq_len-L)),
                                                            0) 
                                                for L in src_len]),
                                    2)
valid_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(
                                                            F.pad(
                                                                torch.ones(L), 
                                                                (0, max_tgt_seq_len-L)),
                                                            0) 
                                                for L in tgt_len]),
                                    2)
valid_cross_pos_matrix = torch.bmm(valid_encoder_pos, valid_decoder_pos.transpose(1, 2)) 
invalid_cross_pos_matrix = 1 - valid_cross_pos_matrix
mask_cross_attention = invalid_cross_pos_matrix.to(torch.bool)
print(mask_cross_attention) 

# 后面计算好相似度矩阵后，再对 mask_cross_attention 中True 的位置填充负无穷的数字

tensor([[[False, False, False, False,  True],
         [False, False, False, False,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[False, False, False,  True,  True],
         [False, False, False,  True,  True],
         [False, False, False,  True,  True],
         [False, False, False,  True,  True],
         [ True,  True,  True,  True,  True]]])


### Decoder self-attention mask

此处的相似度矩阵 shape 为 [batch_size, tgt_seq_len, tgt_seq_len]
因为每个样本的 tgt_seq_len 不一样，所以每个样本的mask是不一样的，但每个mask都是

将相似度矩阵的上三角填充为 负无穷。

In [105]:
valid_decoder_tri_martrix = torch.cat([torch.unsqueeze(
                                                        F.pad(
                                                                torch.tril(torch.ones((L, L))), 
                                                                (0, max_tgt_seq_len-L, max_tgt_seq_len-L, 0)),
                                                                0) 
                                                for L in tgt_len])
invalid_decoder_tri_martrix = 1 - valid_decoder_tri_martrix
invalid_decoder_tri_martrix = invalid_decoder_tri_martrix.to(torch.bool)
print(invalid_decoder_tri_martrix)

tensor([[[ True,  True,  True,  True,  True],
         [False,  True,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False,  True,  True],
         [False, False, False, False,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [False,  True,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False,  True,  True]]])
